In [1]:
import numpy as np
import pandas as pd

class DecisionTreeClassifier:
    def __init__(self, max_depth=5, min_cnt=10):
        self.max_depth = max_depth
        self.min_cnt = min_cnt
        self.tree = None

    def calculate_gini_index(self, groups, classes):
        n_instances = float(sum([len(group) for group in groups]))
        gini = 0.0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            gini += (1.0 - score) * (size / n_instances)
        return gini

    def find_best_split(self, dataset):
        class_values = list(set(row[-1] for row in dataset))
        best_index, best_value, best_score, best_groups = np.inf, np.inf, np.inf, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = self.split_dataset(index, row[index], dataset)
                gini = self.calculate_gini_index(groups, class_values)
                if gini < best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], gini, groups
        return {'index':best_index, 'value':best_value, 'groups':best_groups}

    def split_dataset(self, index, value, dataset):
        left, right = [], []
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right
        
    def build_decision_tree(self, node, max_depth, min_cnt, depth):
        left, right = node['groups']
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right)
            return
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
            return
        if len(left) <= min_cnt:
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.find_best_split(left)
            self.build_decision_tree(node['left'], max_depth, min_cnt, depth+1)
        if len(right) <= min_cnt:
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.find_best_split(right)
            self.build_decision_tree(node['right'], max_depth, min_cnt, depth+1)

    def to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    def train(self, X_train, y_train):
        dataset = np.column_stack((X_train, y_train))
        self.tree = self.find_best_split(dataset)
        self.build_decision_tree(self.tree, self.max_depth, self.min_cnt, 1)

    def predict_single(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict_single(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict_single(node['right'], row)
            else:
                return node['right']

    def predict(self, X_test):
        predictions = []
        for row in X_test:
            predictions.append(self.predict_single(self.tree, row))
        return predictions

def split_train_test(X, y, test_size=0.3):
    split_index = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    return X_train, X_test, y_train, y_test

def load_dataset():
    data = pd.read_csv("data_banknote_authentication.csv", delimiter=',')
    data = np.array(data)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

def calculate_accuracy(y_true, y_pred):
    correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    return correct_predictions / float(len(y_true))

# Main code
if __name__ == "__main__":
    # Load dataset
    X, y = load_dataset()

    # Split dataset into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Initialize and train decision tree classifier
    dt_classifier = DecisionTreeClassifier(max_depth=5, min_cnt=10)
    dt_classifier.train(X_train, y_train)

    # Make predictions on test set
    y_pred_test = dt_classifier.predict(X_test)
    y_pred_train = dt_classifier.predict(X_train)

    # Calculate accuracy
    test_accuracy = calculate_accuracy(y_test, y_pred_test)
    print("Test accuracy:", test_accuracy)
#     train_accuracy = calculate_accuracy(y_train, y_pred_train)
#     print("Train accuracy:", train_accuracy)

Test accuracy: 0.9538834951456311
